In [153]:
%cd "C:\Users\Patrickn\Jupyter_notebooks\Graduation\kokoro"

C:\Users\Patrickn\Jupyter_notebooks\Graduation\kokoro


# init

In [154]:
from kokoro import KPipeline, KModel
from IPython.display import display, Audio
import soundfile as sf
import torch
import torchaudio
from datasets import Dataset, Audio
from transformers import AutoProcessor, SpeechT5HifiGan
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torchaudio import load as load_audio
from transformers import get_scheduler
from tqdm import tqdm
import os
import pandas as pd
import librosa

In [155]:
pipeline = KPipeline(lang_code='a')
text = 'Hello, My name is Patrick'

c:\ProgramData\anaconda\envs\kokoro\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
c:\ProgramData\anaconda\envs\kokoro\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [156]:
generator = pipeline(text, voice='am_puck')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    #display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)

0 Hello, My name is Patrick həlˈO, mˌI nˈAm ɪz pˈætɹɪk


# Pre-proccess data

In [157]:
csv_path = "C:/Users/Patrickn/Jupyter_notebooks/Graduation/Dataset/index.csv"  # Replace with the actual path
audio_folder = "C:/Users/Patrickn/Jupyter_notebooks/Graduation/Dataset/sample"    # Replace with the actual path
df = pd.read_csv(csv_path)
df.head()

,audio_file,text,gender
0,D6x81u_1.mp3,خليني ابدا الفيديو ده وانا بقول ان من,male
1,6f5WmH_2.mp3,ااا ملاحظتي الشخصيه استنتجت ان الستات ما,male
2,abSOwC_3.mp3,بقاش عندها دم,male
3,qqiArB_4.mp3,وان الستات بداوا يستغلوا الرجاله وان,male
4,afzWCf_5.mp3,الستات دول,male


In [158]:
%cd C:/Users/Patrickn/Jupyter_notebooks/Graduation/Dataset/sample

C:\Users\Patrickn\Jupyter_notebooks\Graduation\Dataset\sample


In [159]:
filenames = os.listdir("C:/Users/Patrickn/Jupyter_notebooks/Graduation/Dataset/sample")

filenames = [f for f in os.listdir() if os.path.isfile(f)]
len(filenames)

1000

In [160]:
%cd C:/Users/Patrickn/Jupyter_notebooks/Graduation/kokoro

C:\Users\Patrickn\Jupyter_notebooks\Graduation\kokoro


In [161]:
# Filter rows where the 'filename' column matches one in the list
filtered_df = df[df['audio_file'].isin(filenames)]

# Save the filtered CSV (optional)
filtered_df.to_csv("filtered_dataset.csv", index=False)

In [162]:
csv_path = "C:/Users/Patrickn/Jupyter_notebooks/Graduation/Dataset/filtered_dataset.csv"  # Replace with the actual path
df = pd.read_csv(csv_path)
df.head()

,audio_file,text,gender
0,04HTH0_314.mp3,أحيانًا، لو لا قدّر الله، حصل اصطدام،,male
1,00x68G_697.mp3,وكده عشان انت بتحط في مواقف انك تنزل,male
2,04m0cQ_852.mp3,ومشينا,male
3,027PYP_1114.mp3,كل حاجه مره بقى عربي سول,male
4,06UK7q_276.mp3,فمثلًا، كان الحُكام والإقطاعيين من عاداتهم,male


In [163]:
# Assuming your CSV has columns 'filename' and 'text'
df['audio_file'] = df['audio_file'].apply(lambda x: os.path.join(audio_folder, x))
#dataset = Dataset.from_pandas(df)

In [164]:
dataset = dataset.cast_column("audio_file", Audio(sampling_rate=16000)) # Assuming a target sampling rate of 16000 Hz

In [186]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, target_length):
        self.dataframe = dataframe
        self.target_length = target_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        audio_file = row['audio_file']
        text = row['text']
        audio_tensor = load_audio(audio_file)[0]

        # Pad or truncate audio to the target length
        if audio_tensor.size(1) > self.target_length:
            audio_tensor = audio_tensor[:, :self.target_length]
        else:
            padding = self.target_length - audio_tensor.size(1)
            audio_tensor = torch.nn.functional.pad(audio_tensor, (0, padding))

        return {
            'audio_file': audio_tensor,
            'text': text
        }

class SequentialDataset(Dataset):
    def __init__(self, dataframe, target_length):
        self.dataframe = dataframe
        self.target_length = target_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        audio_file = row['audio_file']
        text = row['text']
        audio_tensor = load_audio(audio_file)[0]

        # Pad or truncate audio to the target length
        if audio_tensor.size(1) > self.target_length:
            audio_tensor = audio_tensor[:, :self.target_length]
        else:
            padding = self.target_length - audio_tensor.size(1)
            audio_tensor = torch.nn.functional.pad(audio_tensor, (0, padding))

        # Convert text from string to tensor of Unicode codepoints
        text_tensor = torch.tensor([ord(c) for c in text], dtype=torch.float32)

        return {
            'audio_file': audio_tensor,
            'text': text_tensor
        }

# Convert the DataFrame to a sequential dataset
sequential_dataset = SequentialDataset(df, target_length=target_length)

In [187]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    audio_list = [item['audio_file'] for item in batch]
    text_list = []
    for item in batch:
        text = item['text']
        if isinstance(text, str):
            text_tensor = torch.tensor([ord(c) for c in text], dtype=torch.float32)
        elif not torch.is_tensor(text):
            # Convert lists (or other iterables) to tensor
            text_tensor = torch.tensor(text, dtype=torch.float32)
        else:
            text_tensor = text
        text_list.append(text_tensor)

    padded_audio = pad_sequence(audio_list, batch_first=True)
    padded_text = pad_sequence(text_list, batch_first=True)
    return {
        'audio_file': padded_audio.to(device),
        'text': padded_text.to(device)
    }

# Create DataLoader using the custom collate function
data_loader = DataLoader(sequential_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# FineTune

In [188]:
# Load the pre-trained model
model = KModel(repo_id='hexgrad/Kokoro-82M')

# Move model to the appropriate device (GPU is highly recommended for finetuning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

KModel(
  (bert): CustomAlbert(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(178, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
            

In [189]:
# Define unknown variables
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 16
num_epochs = 10
target_length = 16000
total_loss = 0

# Define loss functions
criterion_duration = torch.nn.CrossEntropyLoss()
criterion_mel = torch.nn.MSELoss()

# Define optimizer
optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)

In [190]:
# Iterate through the dataset sequentially
for idx in range(len(sequential_dataset)):
    sample = sequential_dataset[idx]
    audio_tensor = sample['audio_file']
    text = sample['text']

    # Process each sample individually
    print(f"Processing sample {idx + 1}/{len(sequential_dataset)}")
    # Add your processing logic here

Processing sample 1/1000
Processing sample 2/1000
Processing sample 3/1000
Processing sample 4/1000
Processing sample 5/1000
Processing sample 6/1000
Processing sample 7/1000
Processing sample 8/1000
Processing sample 9/1000
Processing sample 10/1000
Processing sample 11/1000
Processing sample 12/1000
Processing sample 13/1000
Processing sample 14/1000
Processing sample 15/1000
Processing sample 16/1000
Processing sample 17/1000
Processing sample 18/1000
Processing sample 19/1000
Processing sample 20/1000
Processing sample 21/1000
Processing sample 22/1000
Processing sample 23/1000
Processing sample 24/1000
Processing sample 25/1000
Processing sample 26/1000
Processing sample 27/1000
Processing sample 28/1000
Processing sample 29/1000
Processing sample 30/1000
Processing sample 31/1000
Processing sample 32/1000
Processing sample 33/1000
Processing sample 34/1000
Processing sample 35/1000
Processing sample 36/1000
Processing sample 37/1000
Processing sample 38/1000
Processing sample 39/

In [191]:
# Create DataLoader and optimizer for Kokoro TTS
from torch.utils.data import DataLoader
from torch.optim import AdamW

batch_size = 16
data_loader = DataLoader(processed_dataset, batch_size=batch_size, shuffle=True)

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

criterion_duration = torch.nn.CrossEntropyLoss()
criterion_mel = torch.nn.MSELoss()

In [199]:
from tqdm import tqdm

# Training loop with dynamic padding for variable-length data
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    for batch in progress_bar:
        audio_batch = batch['audio_file']
        text_batch = batch['text']
        # Convert text_batch to tensor if it's a list of strings
        if isinstance(text_batch, list):
            text_tensors = [torch.tensor([ord(c) for c in txt], dtype=torch.float32) for txt in text_batch]
            text_batch = torch.nn.utils.rnn.pad_sequence(text_tensors, batch_first=True).to(device)

        # Forward pass
        outputs = model(audio_batch, text_batch)

        # Compute losses
        loss_duration = criterion_duration(outputs['duration'], text_batch)
        loss_mel = criterion_mel(outputs['mel'], audio_batch)
        loss = loss_duration + loss_mel

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1} completed with average loss: {running_loss / len(data_loader):.4f}")

print("Fine-tuning complete.")

RuntimeError: The expanded size of the tensor (1) must match the existing size (16) at non-singleton dimension 1.  Target sizes: [2, 1, -1].  Tensor sizes: [16, 0]